In [1]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install datasets
!pip install sentencepiece
!pip install transformers==4.21.2
!pip install tokenizers==0.12.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!nvidia-smi

Fri Nov 25 06:41:57 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    45W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
# ====================================================
# CFG
# ====================================================
class CFG:
    debug=False
    apex=True
    print_freq=20
    num_workers=4
    model="microsoft/deberta-v3-base"
    # model='microsoft/deberta-base'  △
    # model='roberta-base'  
    # model='roberta-large'  x
    # model='roberta-large-mnli'
    # model='google/bigbird-roberta-base'
    # model='google/bigbird-roberta-large'
    # model='xlnet-large-cased'  △
    # model='albert-xxlarge-v2'
    # model="microsoft/deberta-large" o
    # model="microsoft/deberta-v3-large"  o
    # model='microsoft/deberta-v2-xlarge' x
    # model='microsoft/deberta-v2-xxlarge'
    # model='microsoft/deberta-xlarge' 
    # model='funnel-transformer/large' o
    # model='funnel-transformer/medium' △
    # model='albert-base-v2'
    # model='albert-large-v2'  x
    # model='google/electra-large-discriminator'  x
    # model='google/electra-base-discriminator'  x
    # model="facebook/bart-large-mnli"
    # model="facebook/bart-large"  o
    # model="facebook/bart-base"
    # model = "distilbert-base-uncased" x
    # model = "allenai/longformer-large-4096" x
    # model = "allenai/longformer-base-4096"
    # model = "uw-madison/yoso-4096"  x
    # model = "xlm-roberta-large"
    # model = "xlm-roberta-base"
    # model = "google/muril-large-cased" x
    # model = "google/rembert" x
    gradient_checkpointing=True
    scheduler='polynomial' # ['linear', 'cosine']
    batch_scheduler=True
    num_cycles=0.5
    num_warmup_steps=0
    epochs=4
    encoder_lr=2e-5
    decoder_lr=2e-5
    min_lr=1e-6
    eps=1e-6
    betas=(0.9, 0.999)
    batch_size=8
    use_prior_wd = True
    use_bertadam = False
    max_len=800
    weight_decay=0.01
    gradient_accumulation_steps=1
    max_grad_norm=1000
    target_size=6
    fc_dropout=0.2
    target_cols=['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']
    seed=42
    n_fold=5
    trn_fold=[0, 1, 2, 3, 4]
    train=True
    freezing=False
    mixout = 0.5
    is_mixout = True
    num_reinit_layers = 1
    is_reinit_layer = True
    fgm = True
    
if CFG.debug:
    CFG.epochs = 2
    CFG.trn_fold = [0]

In [5]:
import os

DIR = '/content/drive/MyDrive/Competitions/Kaggle/FeedBack3'
INPUT_DIR = os.path.join(DIR,'input')
OUTPUT_DIR = os.path.join(DIR,'output')
OUTPUT_MODEL_DIR = DIR + '/output/EXP015/'
if not os.path.exists(OUTPUT_MODEL_DIR):
    os.makedirs(OUTPUT_MODEL_DIR)

In [6]:
# ====================================================
# Library
# ====================================================
from google.colab import runtime
import gc
import re
import ast
import sys
import copy
import json
import time
import math
import string
import pickle
import random
import joblib
import itertools
import warnings
warnings.filterwarnings("ignore")

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

from text_unidecode import unidecode
from typing import Dict, List, Tuple
import codecs


import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW, Optimizer
from torch.utils.data import DataLoader, Dataset
from torch.autograd.function import InplaceFunction 
import torch.nn.init as init

#os.system('pip uninstall -y transformers')
#os.system('pip uninstall -y tokenizers')
#os.system('python -m pip install --no-index --find-links=/content/drive/MyDrive/Competitions/Kaggle/FeedBack3/pip_wheel.ipynb transformers')
#os.system('python -m pip install --no-index --find-links=/content/drive/MyDrive/Competitions/Kaggle/FeedBack3/pip_wheel.ipynb tokenizers')
import tokenizers
import transformers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup,get_polynomial_decay_schedule_with_warmup
%env TOKENIZERS_PARALLELISM=true

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizers.__version__: 0.12.1
transformers.__version__: 4.21.2
env: TOKENIZERS_PARALLELISM=true


In [7]:
# ====================================================
# Utils
# ====================================================
def MCRMSE(y_trues, y_preds):
    scores = []
    idxes = y_trues.shape[1]
    for i in range(idxes):
        y_true = y_trues[:,i]
        y_pred = y_preds[:,i]
        score = mean_squared_error(y_true, y_pred, squared=False) # RMSE
        scores.append(score)
    mcrmse_score = np.mean(scores)
    return mcrmse_score, scores


def get_score(y_trues, y_preds):
    mcrmse_score, scores = MCRMSE(y_trues, y_preds)
    return mcrmse_score, scores


def get_logger(filename=OUTPUT_MODEL_DIR+'train'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()


def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=CFG.seed)

In [8]:
def freeze(module):
    """
    Freezes module's parameters.
    """
    
    for parameter in module.parameters():
        parameter.requires_grad = False
        
def get_freezed_parameters(module):
    """
    Returns names of freezed parameters of the given module.
    """
    
    freezed_parameters = []
    for name, parameter in module.named_parameters():
        if not parameter.requires_grad:
            freezed_parameters.append(name)
            
    return freezed_parameters

def set_embedding_parameters_bits(embeddings_path, optim_bits=32):
    """
    https://github.com/huggingface/transformers/issues/14819#issuecomment-1003427930
    """
    
    embedding_types = ("word", "position", "token_type")
    for embedding_type in embedding_types:
        attr_name = f"{embedding_type}_embeddings"
        
        if hasattr(embeddings_path, attr_name): 
            bnb.optim.GlobalOptimManager.get_instance().register_module_override(
                getattr(embeddings_path, attr_name), 'weight', {'optim_bits': optim_bits}
            )

In [9]:
def replace_encoding_with_utf8(error: UnicodeError) -> Tuple[bytes, int]:
    return error.object[error.start : error.end].encode("utf-8"), error.end


def replace_decoding_with_cp1252(error: UnicodeError) -> Tuple[str, int]:
    return error.object[error.start : error.end].decode("cp1252"), error.end

# Register the encoding and decoding error handlers for `utf-8` and `cp1252`.
codecs.register_error("replace_encoding_with_utf8", replace_encoding_with_utf8)
codecs.register_error("replace_decoding_with_cp1252", replace_decoding_with_cp1252)

def resolve_encodings_and_normalize(text: str) -> str:
    """Resolve the encoding problems and normalize the abnormal characters."""
    text = (
        text.encode("raw_unicode_escape")
        .decode("utf-8", errors="replace_decoding_with_cp1252")
        .encode("cp1252", errors="replace_encoding_with_utf8")
        .decode("utf-8", errors="replace_decoding_with_cp1252")
    )
    text = unidecode(text)
    return text

In [10]:
def del_empty_text(text):
    """ Custom text cleaning. """
    text = text.strip()
    text = text.replace('\n', ' ')
    text = text.replace('\t', ' ')
    text = re.sub(' +', ' ', text)
    text = text.strip()

    return text

In [11]:
# ====================================================
# Data Loading
# ====================================================
train = pd.read_csv(os.path.join(INPUT_DIR,'feedback3_train.csv'))
pseudo_df = pd.read_csv(os.path.join(INPUT_DIR,'feedback3_pseudo_v3base_meanpool.csv'))

print(f"train shape: {train.shape}")
display(train.head(3))
print(f"paseud shape:{pseudo_df.shape}")
display(pseudo_df.head(3))

train shape: (3911, 16)


,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions,fold_a4,fold_a5,Topic,prob,Count,Bertopic,fold_c4,fold_c5
0,0016926B079C,I think that students would benefit from learn...,3.5,3.5,3.0,3.0,4.0,3.0,2,1,0,1.0,298,online classes home students,1,3
1,0022683E9EA5,When a problem is a change you have to let it ...,2.5,2.5,3.0,2.0,2.0,2.5,0,0,44,1.0,20,change world want people,1,1
2,00299B378633,"Dear, Principal\n\nIf u change the school poli...",3.0,3.5,3.0,3.0,3.0,2.5,1,4,31,1.0,36,sports average policy play,3,4


paseud shape:(10000, 14)


,text_id,full_text,fold_a4,fold_a5,topic,topic_name,fold_c4,fold_c5,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,476DAA9251E5,In my opinon I don't think that driverless car...,3,4,3,technology people use use technology,0,0,3.763730,3.657085,3.875090,4.011017,4.031650,3.705348
1,FB9569BDE4BB,My option to this device is it is valueable to...,0,0,3,technology people use use technology,0,0,2.576126,2.624726,2.897718,2.617785,2.692831,2.720349
2,049B62690A5E,The new technology that is called the Facial A...,3,3,3,technology people use use technology,0,0,3.101968,3.301859,3.349278,3.255425,3.310808,2.912409


In [12]:
train['full_text'] = train['full_text'].apply(resolve_encodings_and_normalize)
train['full_text'] = train['full_text'].apply(del_empty_text)
pseudo_df['full_text'] = pseudo_df['full_text'].apply(resolve_encodings_and_normalize)
pseudo_df['full_text'] = pseudo_df['full_text'].apply(del_empty_text)

In [13]:
train['full_text'] = train['Bertopic'] + '[SEP]' + train['full_text']
pseudo_df['full_text'] = pseudo_df['topic_name'] + '[SEP]' + pseudo_df['full_text']

display(train.head(3))
display(pseudo_df.head(3))

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions,fold_a4,fold_a5,Topic,prob,Count,Bertopic,fold_c4,fold_c5
0,0016926B079C,online classes home students[SEP]I think that ...,3.5,3.5,3.0,3.0,4.0,3.0,2,1,0,1.0,298,online classes home students,1,3
1,0022683E9EA5,change world want people[SEP]When a problem is...,2.5,2.5,3.0,2.0,2.0,2.5,0,0,44,1.0,20,change world want people,1,1
2,00299B378633,"sports average policy play[SEP]Dear, Principal...",3.0,3.5,3.0,3.0,3.0,2.5,1,4,31,1.0,36,sports average policy play,3,4


,text_id,full_text,fold_a4,fold_a5,topic,topic_name,fold_c4,fold_c5,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,476DAA9251E5,technology people use use technology[SEP]In my...,3,4,3,technology people use use technology,0,0,3.763730,3.657085,3.875090,4.011017,4.031650,3.705348
1,FB9569BDE4BB,technology people use use technology[SEP]My op...,0,0,3,technology people use use technology,0,0,2.576126,2.624726,2.897718,2.617785,2.692831,2.720349
2,049B62690A5E,technology people use use technology[SEP]The n...,3,3,3,technology people use use technology,0,0,3.101968,3.301859,3.349278,3.255425,3.310808,2.912409


In [14]:
display(train.groupby('fold_c5').size())

fold_c5
0    794
1    778
2    788
3    783
4    768
dtype: int64

In [15]:
display(pseudo_df.groupby('fold_c5').size())

fold_c5
0    2000
1    2000
2    2000
3    2000
4    2000
dtype: int64

In [16]:
if CFG.debug:
    display(train.groupby('fold_c5').size())
    train = train.sample(n=500, random_state=0).reset_index(drop=True)
    display(train.groupby('fold_c5').size())

if CFG.debug:
    pseudo_df = pseudo_df.sample(n=200, random_state=0).reset_index(drop=True)
    display(pseudo_df.groupby('fold_c5').size())

In [17]:
# ====================================================
# tokenizer
# ====================================================
tokenizer = AutoTokenizer.from_pretrained(CFG.model)
#tokenizer.save_pretrained(OUTPUT_MODEL_DIR+'/tokenizer/')
CFG.tokenizer = tokenizer

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [18]:
class FGM():
    def __init__(self, model, eps=1.):
        self.model = model
        self.eps = eps
        self.backup = {}

    def attack(self, emb_name='word_embeddings'):
        for name, param in self.model.named_parameters():
            if param.requires_grad and emb_name in name:
                self.backup[name] = param.data.clone()
                norm = torch.norm(param.grad)
                if norm != 0:
                    r_at = self.eps * param.grad / norm
                    param.data.add_(r_at)

    def restore(self, emb_name='word_embeddings'):
        for name, param in self.model.named_parameters():
            if param.requires_grad and emb_name in name:
                assert name in self.backup
                param.data = self.backup[name]
            self.backup = {}

In [19]:
class Mixout(InplaceFunction):
    @staticmethod
    def _make_noise(input):
        return input.new().resize_as_(input)

    @classmethod
    def forward(cls, ctx, input, target=None, p=0.0, training=False, inplace=False):
        if p < 0 or p > 1:
            raise ValueError("A mix probability of mixout has to be between 0 and 1," " but got {}".format(p))
        if target is not None and input.size() != target.size():
            raise ValueError(
                "A target tensor size must match with a input tensor size {},"
                " but got {}".format(input.size(), target.size())
            )
        ctx.p = p
        ctx.training = training

        if ctx.p == 0 or not ctx.training:
            return input

        if target is None:
            target = cls._make_noise(input)
            target.fill_(0)
        target = target.to(input.device)

        if inplace:
            ctx.mark_dirty(input)
            output = input
        else:
            output = input.clone()

        ctx.noise = cls._make_noise(input)
        if len(ctx.noise.size()) == 1:
            ctx.noise.bernoulli_(1 - ctx.p)
        else:
            ctx.noise[0].bernoulli_(1 - ctx.p)
            ctx.noise = ctx.noise[0].repeat(input.size()[0], 1)
        ctx.noise.expand_as(input)

        if ctx.p == 1:
            output = target
        else:
            output = ((1 - ctx.noise) * target + ctx.noise * output - ctx.p * target) / (1 - ctx.p)
        return output

    @staticmethod
    def backward(ctx, grad_output):
        if ctx.p > 0 and ctx.training:
            return grad_output * ctx.noise, None, None, None, None
        else:
            return grad_output, None, None, None, None


def mixout(input, target=None, p=0.0, training=False, inplace=False):
    return Mixout.apply(input, target, p, training, inplace)


class MixLinear(torch.nn.Module):
    __constants__ = ["bias", "in_features", "out_features"]
    def __init__(self, in_features, out_features, bias=True, target=None, p=0.0):
        super(MixLinear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.Tensor(out_features, in_features))
        if bias:
            self.bias = Parameter(torch.Tensor(out_features))
        else:
            self.register_parameter("bias", None)
        self.reset_parameters()
        self.target = target
        self.p = p

    def reset_parameters(self):
        init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in)
            init.uniform_(self.bias, -bound, bound)

    def forward(self, input):
        return F.linear(input, mixout(self.weight, self.target, self.p, self.training), self.bias)

    def extra_repr(self):
        type = "drop" if self.target is None else "mix"
        return "{}={}, in_features={}, out_features={}, bias={}".format(
            type + "out", self.p, self.in_features, self.out_features, self.bias is not None
        )
    
def replace_mixout(model, mixout_p):
    for sup_module in model.modules():
        for name, module in sup_module.named_children():
            if isinstance(module, nn.Dropout):
                module.p = 0.0
            if isinstance(module, nn.Linear):
                target_state_dict = module.state_dict()
                bias = True if module.bias is not None else False
                new_module = MixLinear(
                    module.in_features, module.out_features, bias, target_state_dict["weight"], mixout_p
                )
                new_module.load_state_dict(target_state_dict)
                setattr(sup_module, name, new_module)
    return model

In [20]:
def reinit_layers(model):

    #for layer in model.model.encoder.layer[-CFG.num_reinit_layers:]:
    for layer in model.encoder.layer[-CFG.num_reinit_layers:]:    #Custome model内(backbone)

            for module in layer.modules():

                if isinstance(module,nn.Linear):
                    module.weight.data.normal_(mean=0.0,std=model.config.initializer_range)
                    if module.bias is not None:
                            module.bias.data.zero_()
                elif isinstance(module, nn.Embedding):
                        module.weight.data.normal_(mean=0.0, std=model.config.initializer_range)
                        if module.padding_idx is not None:
                            module.weight.data[module.padding_idx].zero_()
                elif isinstance(module, nn.LayerNorm):
                        module.bias.data.zero_()
                        module.weight.data.fill_(1.0)
                        
    return model

In [21]:
# ====================================================
# Dataset
# ====================================================
def prepare_input(cfg, text):
    inputs = cfg.tokenizer.encode_plus(
        text, 
        return_tensors=None, 
        add_special_tokens=True, 
        max_length=CFG.max_len,
        pad_to_max_length=True,
        truncation=True
    )
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.int32)
    return inputs


class TrainDataset(Dataset):
    def __init__(self, cfg, df):
        self.cfg = cfg
        self.texts = df['full_text'].values
        self.labels = df[cfg.target_cols].values

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        inputs = prepare_input(self.cfg, self.texts[item])
        label = torch.tensor(self.labels[item], dtype=torch.float32)
        return inputs, label
        
class TestDataset(Dataset):
    def __init__(self, cfg, df):
        self.cfg = cfg
        self.texts = df['full_text'].values


    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        inputs = prepare_input(self.cfg, self.texts[item])
        return inputs    

def collate(inputs):
    mask_len = int(inputs["attention_mask"].sum(axis=1).max())
    for k, v in inputs.items():
        inputs[k] = inputs[k][:,:mask_len]
    return inputs

In [22]:
# ====================================================
# Model
# ====================================================
class MeanPooling(nn.Module):
    def __init__(self):
        super(MeanPooling, self).__init__()
        
    def forward(self, last_hidden_state, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        return mean_embeddings

class CustomModel(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
            self.config.hidden_dropout = 0.
            self.config.hidden_dropout_prob = 0.
            self.config.attention_dropout = 0.
            self.config.attention_probs_dropout_prob = 0.
            LOGGER.info(self.config)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
        else:
            self.model = AutoModel.from_config(self.config)
        if cfg.is_mixout:
            self.model = replace_mixout(self.model, CFG.mixout)
            print(f"Initialized Mixout (p={CFG.mixout}) Regularization")
        if CFG.is_reinit_layer:
            self.model = reinit_layers(self.model)
            print(f'Reinitializing Last {CFG.num_reinit_layers} Layers.')
        if self.cfg.gradient_checkpointing:
            self.model.gradient_checkpointing_enable()
            
        # Freezing
        if cfg.freezing:
            # freezing embeddings and first 2 layers of encoder
            freeze((self.model).embeddings)
            freeze((self.model).encoder.layer[:2])
            cfg.after_freezed_parameters = filter(lambda parameter: parameter.requires_grad, (self.model).parameters())
            
        if 'microsoft/deberta-xlarge' in CFG.model:
            self.model.embeddings.requires_grad_(False)
            self.model.encoder.layer[:24].requires_grad_(False)
        if 'microsoft/deberta-v2-xlarge' in CFG.model:
            self.model.embeddings.requires_grad_(False)
            self.model.encoder.layer[:12].requires_grad_(False)
            

        self.high_dropout = nn.Dropout(p=0.5)
        
        self.mean_pooler = MeanPooling()
        

        self.fc = nn.Sequential(
            nn.Linear(self.config.hidden_size, CFG.target_size)
        )
        self.layer_norm1 = nn.LayerNorm(self.config.hidden_size,eps=1e-5)
        
        self._init_weights(self.fc)
        self._init_weights(self.layer_norm1)

        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        
    def feature(self, inputs):
        outputs = self.model(**inputs)
        last_hidden_states = outputs[0]
        
        mean_pool = self.mean_pooler(last_hidden_states, inputs['attention_mask'])

        feature = self.layer_norm1(mean_pool)
        return feature, outputs
    

    def forward(self, inputs=None, labels=None):
        feature,outputs = self.feature(inputs)
        logits = torch.mean(
            torch.stack(
                [self.fc(self.high_dropout(feature)) for _ in range(5)],
                dim=0,
            ),
            dim=0,
        )

        # calculate loss
        loss = None
        if labels is not None:
            loss_fn = nn.SmoothL1Loss(reduction='mean')
            loss = loss_fn(logits, labels)
        
        output = (logits,) + outputs[2:]
        return ((loss,) + output) if loss is not None else output

In [23]:
# ====================================================
# Loss
# ====================================================
class RMSELoss(nn.Module):
    def __init__(self, reduction='mean', eps=1e-9):
        super().__init__()
        self.mse = nn.MSELoss(reduction='none')
        self.reduction = reduction
        self.eps = eps

    def forward(self, y_pred, y_true):
        loss = torch.sqrt(self.mse(y_pred, y_true) + self.eps)
        if self.reduction == 'none':
            loss = loss
        elif self.reduction == 'sum':
            loss = loss.sum()
        elif self.reduction == 'mean':
            loss = loss.mean()
        return loss

In [24]:
class PriorWD(Optimizer):
    def __init__(self, optim, use_prior_wd=False, exclude_last_group=True):
        super(PriorWD, self).__init__(optim.param_groups, optim.defaults)
        self.param_groups = optim.param_groups
        self.optim = optim
        self.use_prior_wd = use_prior_wd
        self.exclude_last_group = exclude_last_group
        self.weight_decay_by_group = []
        for i, group in enumerate(self.param_groups):
            self.weight_decay_by_group.append(group["weight_decay"])
            group["weight_decay"] = 0

        self.prior_params = {}
        for i, group in enumerate(self.param_groups):
            for p in group["params"]:
                self.prior_params[id(p)] = p.detach().clone()

    def step(self, closure=None):
        if self.use_prior_wd:
            for i, group in enumerate(self.param_groups):
                for p in group["params"]:
                    if self.exclude_last_group and i == len(self.param_groups):
                        p.data.add_(-group["lr"] * self.weight_decay_by_group[i], p.data)
                    else:
                        p.data.add_(
                            -group["lr"] * self.weight_decay_by_group[i], p.data - self.prior_params[id(p)],
                        )
        loss = self.optim.step(closure)

        return loss

    def compute_distance_to_prior(self, param):
        assert id(param) in self.prior_params, "parameter not in PriorWD optimizer"
        return (param.data - self.prior_params[id(param)]).pow(2).sum().sqrt()

In [25]:
def calculate_loss(inputs, labels, model, criterion, is_valid=True, device="cpu"):    
    outputs = model(inputs,labels)
    loss, logits = outputs[:2]
    return (loss, logits) if is_valid else loss

In [26]:
# ====================================================
# Helper functions
# ====================================================
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))


def train_fn(fold, train_loader, model, criterion, optimizer, epoch, scheduler, device, fgm):
    model.zero_grad()
    model.train()
    #awp_start = 1.0
    #optimizer.zero_grad()
    scaler = torch.cuda.amp.GradScaler(enabled=CFG.apex)
    losses = AverageMeter()
    start = end = time.time()
    global_step = 0
    for step, (inputs, labels) in enumerate(train_loader):
        inputs = collate(inputs)
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        #if epoch >= awp_start:
        #    awp.perturb(inputs, labels, criterion)
        with torch.cuda.amp.autocast(enabled=CFG.apex):
             loss = calculate_loss(inputs=inputs, labels=labels, model=model, criterion=criterion, is_valid=False, device=device)
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        losses.update(loss.item(), batch_size)
        scaler.scale(loss).backward()
        if scaler is not None:
            scaler.unscale_(optimizer)
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_grad_norm)
        
        #awp.restore()
        if CFG.fgm:
          fgm.attack() 
          adversarial_loss = calculate_loss(inputs=inputs, labels=labels, model=model, criterion=criterion, is_valid=False, device=device)
          scaler.scale(adversarial_loss).backward()
          fgm.restore()
              
        if (step + 1) % CFG.gradient_accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            global_step += 1
            if CFG.batch_scheduler:
                scheduler.step()
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(train_loader)-1):
            print('Epoch: [{0}][{1}/{2}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  #'Grad: {grad_norm:.4f}  '
                  'LR: {lr:.8f}  '
                  .format(epoch+1, step, len(train_loader), 
                          remain=timeSince(start, float(step+1)/len(train_loader)),
                          loss=losses,
                          #grad_norm=grad_norm,
                          lr=scheduler.get_lr()[0]))
            
    return losses.avg


def valid_fn(valid_loader, model, criterion, device):
    losses = AverageMeter()
    model.eval()
    preds = []
    start = end = time.time()
    for step, (inputs, labels) in enumerate(valid_loader):
        inputs = collate(inputs)
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        with torch.no_grad():
            loss, y_preds = calculate_loss(inputs=inputs, labels=labels, model=model, criterion=criterion, is_valid=True, device=device)
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        losses.update(loss.item(), batch_size)
        preds.append(y_preds.to('cpu').numpy())
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(valid_loader)-1):
            print('EVAL: [{0}/{1}] '
                    'Elapsed {remain:s} '
                    'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                    .format(step, len(valid_loader),
                            loss=losses,
                            remain=timeSince(start, float(step+1)/len(valid_loader))))
    predictions = np.concatenate(preds)
    return losses.avg, predictions

def pseudo_fn(pseudo_loader, model, device, cfg):
    preds = []
    model.eval()
    model.to(device)
    for step, (inputs, labels) in enumerate(pseudo_loader):
        inputs = collate(inputs)
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        with torch.no_grad():
            y_preds = model(inputs)
            y_preds = y_preds[0]
        preds.append(y_preds.to('cpu').numpy())
    predictions = np.concatenate(preds)
    return predictions

In [27]:
# ====================================================
# train loop
# ====================================================
def train_loop(folds, p_folds, fold):
    
    LOGGER.info(f"========== fold: {fold} training ==========")

    # ====================================================
    # loader
    # ====================================================
    train_folds = folds[folds['fold_c5'] != fold].reset_index(drop=True)
    valid_folds = folds[folds['fold_c5'] == fold].reset_index(drop=True)
    valid_labels = valid_folds[CFG.target_cols].values

    pseudo_folds = p_folds[p_folds['fold_c5']==fold].reset_index(drop=True)
    
    valid_dataset = TrainDataset(CFG, valid_folds)

    valid_loader = DataLoader(valid_dataset,
                              batch_size=CFG.batch_size * 2,
                              shuffle=False,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

    # ====================================================
    # model & optimizer
    # ====================================================
    model = CustomModel(CFG, config_path=None, pretrained=True)
    torch.save(model.config, OUTPUT_MODEL_DIR+'config.pth')
    model.to(device)
    
    def get_optimizer_params(model, encoder_lr=5e-6, decoder_lr=1e-4, weight_decay=0.0):
        param_optimizer = list(model.named_parameters())
        no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
        group1=['layer.0.','layer.1.','layer.2.','layer.3.']
        group2=['layer.4.','layer.5.','layer.6.','layer.7.']    
        group3=['layer.8.','layer.9.','layer.10.','layer.11.']
        group_all=['layer.0.','layer.1.','layer.2.','layer.3.','layer.4.','layer.5.','layer.6.','layer.7.','layer.8.','layer.9.','layer.10.','layer.11.']
        optimizer_parameters1 = [
            {'params': [p for n, p in model.model.named_parameters() if not any(nd in n for nd in no_decay)],
             'lr': encoder_lr, 'weight_decay': weight_decay},
            {'params': [p for n, p in model.model.named_parameters() if any(nd in n for nd in no_decay)],
             'lr': encoder_lr, 'weight_decay': 0.0},
            {'params': [p for n, p in model.named_parameters() if "model" not in n],
             'lr': decoder_lr, 'weight_decay': 0.0}
        ]
        optimizer_parameters2 = [
        {'params': [p for n, p in model.model.named_parameters() if not any(nd in n for nd in no_decay) and not any(nd in n for nd in group_all)],'weight_decay': weight_decay},
        {'params': [p for n, p in model.model.named_parameters() if not any(nd in n for nd in no_decay) and any(nd in n for nd in group1)],'weight_decay': weight_decay, 'lr': encoder_lr/2.6},
        {'params': [p for n, p in model.model.named_parameters() if not any(nd in n for nd in no_decay) and any(nd in n for nd in group2)],'weight_decay': weight_decay, 'lr': encoder_lr},
        {'params': [p for n, p in model.model.named_parameters() if not any(nd in n for nd in no_decay) and any(nd in n for nd in group3)],'weight_decay': weight_decay, 'lr': encoder_lr*2.6},
        {'params': [p for n, p in model.model.named_parameters() if any(nd in n for nd in no_decay) and not any(nd in n for nd in group_all)],'weight_decay': 0.0},
        {'params': [p for n, p in model.model.named_parameters() if any(nd in n for nd in no_decay) and any(nd in n for nd in group1)],'weight_decay': 0.0, 'lr': encoder_lr/2.6},
        {'params': [p for n, p in model.model.named_parameters() if any(nd in n for nd in no_decay) and any(nd in n for nd in group2)],'weight_decay': 0.0, 'lr': encoder_lr},
        {'params': [p for n, p in model.model.named_parameters() if any(nd in n for nd in no_decay) and any(nd in n for nd in group3)],'weight_decay': 0.0, 'lr': encoder_lr*2.6},
        {'params': [p for n, p in model.named_parameters() if "model" not in n], 'lr':decoder_lr, "momentum" : 0.99},
    ]
        return optimizer_parameters2

    optimizer_parameters = get_optimizer_params(model,
                                                encoder_lr=CFG.encoder_lr, 
                                                decoder_lr=CFG.decoder_lr,
                                                weight_decay=CFG.weight_decay)
    optimizer = AdamW(optimizer_parameters, lr=CFG.encoder_lr, eps=CFG.eps, betas=CFG.betas)
    optimizer = PriorWD(optimizer, use_prior_wd=CFG.use_prior_wd)
    
    # ====================================================
    # scheduler
    # ====================================================
    def get_scheduler(cfg, optimizer, num_train_steps):
        if cfg.scheduler == 'linear':
            scheduler = get_linear_schedule_with_warmup(
                optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps
            )
        elif cfg.scheduler == 'cosine':
            scheduler = get_cosine_schedule_with_warmup(
                optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps, num_cycles=cfg.num_cycles
            )
        elif cfg.scheduler == 'polynomial':
            warmup_steps = int(len(train_folds) / CFG.batch_size * 0.1)
            scheduler = get_polynomial_decay_schedule_with_warmup(
                optimizer, warmup_steps, num_train_steps, lr_end=7e-7, power=3.0)
        return scheduler
    
    num_train_steps = int(len(train_folds) / CFG.batch_size * CFG.epochs)
    scheduler = get_scheduler(CFG, optimizer, num_train_steps)

    # ====================================================
    # loop
    # ====================================================
    criterion = nn.SmoothL1Loss(reduction='mean') # RMSELoss(reduction="mean")

    print('Enable FGM')
    fgm = FGM(model=model, eps=0.1)

    best_score = np.inf

    for epoch in range(CFG.epochs):

        pseudo_dataset = TrainDataset(CFG, pseudo_folds)
        pseudo_loader = DataLoader(pseudo_dataset,
                              batch_size=CFG.batch_size * 2,
                              shuffle=False,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

        df_folds = pd.concat([train_folds[["text_id","full_text","cohesion","syntax","vocabulary","phraseology","grammar","conventions"]],
                                pseudo_folds[["text_id","full_text","cohesion","syntax","vocabulary","phraseology","grammar","conventions"]]]).reset_index(drop=True)
        train_dataset = TrainDataset(CFG, df_folds)
        train_loader = DataLoader(train_dataset,
                              batch_size=CFG.batch_size,
                              shuffle=True,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=True)

        start_time = time.time()

        # train
        avg_loss = train_fn(fold, train_loader, model, criterion, optimizer, epoch, scheduler, device, fgm)

        # eval
        avg_val_loss, predictions = valid_fn(valid_loader, model, criterion, device)

        # pseudo pred
        pseudo_predictions = pseudo_fn(pseudo_loader, model, device, CFG)
        
        # scoring
        score, scores = get_score(valid_labels, predictions)

        elapsed = time.time() - start_time

        LOGGER.info(f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s')
        LOGGER.info(f'Epoch {epoch+1} - Score: {score:.4f}  Scores: {scores}')
        
        if best_score > score:
            best_score = score
            LOGGER.info(f'Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model')
            torch.save({'model': model.state_dict(),
                        'predictions': predictions},
                        OUTPUT_MODEL_DIR+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth")
            
        if epoch!=3:
            pseudo_folds[CFG.target_cols] = pseudo_predictions


    predictions = torch.load(OUTPUT_MODEL_DIR+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth", 
                             map_location=torch.device('cpu'))['predictions']
    valid_folds[[f"pred_{c}" for c in CFG.target_cols]] = predictions

    torch.cuda.empty_cache()
    gc.collect()
    
    return valid_folds

In [ ]:
if __name__ == '__main__':
    
    def get_result(oof_df):
        labels = oof_df[CFG.target_cols].values
        preds = oof_df[[f"pred_{c}" for c in CFG.target_cols]].values
        score, scores = get_score(labels, preds)
        LOGGER.info(f'Score: {score:<.4f}  Scores: {scores}')
    
    if CFG.train:
        oof_df = pd.DataFrame()
        for fold in range(CFG.n_fold):
            if fold in CFG.trn_fold:
                _oof_df = train_loop(train,pseudo_df, fold)
                oof_df = pd.concat([oof_df, _oof_df])
                LOGGER.info(f"========== fold: {fold} result ==========")
                get_result(_oof_df)
        oof_df = oof_df.reset_index(drop=True)
        LOGGER.info(f"========== CV ==========")
        get_result(oof_df)
        oof_df.to_pickle(OUTPUT_MODEL_DIR+'oof_df.pkl')

========== fold: 0 training ==========
INFO:__main__:========== fold: 0 training ==========
DebertaV2Config {
  "_name_or_path": "microsoft/deberta-v3-base",
  "attention_dropout": 0.0,
  "attention_probs_dropout_prob": 0.0,
  "hidden_act": "gelu",
  "hidden_dropout": 0.0,
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta-v2",
  "norm_rel_ebd": "layer_norm",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 768,
  "pos_att_type": [
    "p2c",
    "c2p"
  ],
  "position_biased_input": false,
  "position_buckets": 256,
  "relative_attention": true,
  "share_att_key": true,
  "transformers_version": "4.21.2",
  "type_vocab_size": 0,
  "vocab_size": 128100
}

INFO:__main__:Deberta

Initialized Mixout (p=0.5) Regularization
Reinitializing Last 1 Layers.
Enable FGM
Epoch: [1][0/639] Elapsed 0m 3s (remain 35m 34s) Loss: 2.5354(2.5354) LR: 0.00000053  
Epoch: [1][20/639] Elapsed 0m 31s (remain 15m 18s) Loss: 0.3502(1.6509) LR: 0.00001105  
Epoch: [1][40/639] Elapsed 0m 58s (remain 14m 6s) Loss: 0.0996(0.9396) LR: 0.00001989  
Epoch: [1][60/639] Elapsed 1m 26s (remain 13m 35s) Loss: 0.1793(0.6795) LR: 0.00001914  
Epoch: [1][80/639] Elapsed 1m 52s (remain 12m 58s) Loss: 0.0964(0.5430) LR: 0.00001841  
Epoch: [1][100/639] Elapsed 2m 21s (remain 12m 35s) Loss: 0.1207(0.4622) LR: 0.00001770  
Epoch: [1][120/639] Elapsed 2m 48s (remain 12m 1s) Loss: 0.1813(0.4039) LR: 0.00001701  
Epoch: [1][140/639] Elapsed 3m 16s (remain 11m 33s) Loss: 0.0907(0.3635) LR: 0.00001634  
Epoch: [1][160/639] Elapsed 3m 43s (remain 11m 4s) Loss: 0.0793(0.3319) LR: 0.00001568  
Epoch: [1][180/639] Elapsed 4m 10s (remain 10m 33s) Loss: 0.1512(0.3092) LR: 0.00001505  
Epoch: [1][200/639] Elapsed

Epoch 1 - avg_train_loss: 0.1626  avg_val_loss: 0.1062  time: 945s
INFO:__main__:Epoch 1 - avg_train_loss: 0.1626  avg_val_loss: 0.1062  time: 945s
Epoch 1 - Score: 0.4618  Scores: [0.4951324651259908, 0.4755990078335953, 0.42090843337036477, 0.4651627592924639, 0.46454338783038607, 0.44921942809992393]
INFO:__main__:Epoch 1 - Score: 0.4618  Scores: [0.4951324651259908, 0.4755990078335953, 0.42090843337036477, 0.4651627592924639, 0.46454338783038607, 0.44921942809992393]
Epoch 1 - Save Best Score: 0.4618 Model
INFO:__main__:Epoch 1 - Save Best Score: 0.4618 Model


Epoch: [2][0/639] Elapsed 0m 1s (remain 17m 17s) Loss: 0.1207(0.1207) LR: 0.00000495  
Epoch: [2][20/639] Elapsed 0m 29s (remain 14m 17s) Loss: 0.1287(0.1031) LR: 0.00000468  
Epoch: [2][40/639] Elapsed 0m 57s (remain 13m 52s) Loss: 0.0926(0.1034) LR: 0.00000442  
Epoch: [2][60/639] Elapsed 1m 25s (remain 13m 31s) Loss: 0.1048(0.1031) LR: 0.00000417  
Epoch: [2][80/639] Elapsed 1m 54s (remain 13m 7s) Loss: 0.0602(0.1032) LR: 0.00000393  
Epoch: [2][100/639] Elapsed 2m 22s (remain 12m 39s) Loss: 0.0783(0.1028) LR: 0.00000371  
Epoch: [2][120/639] Elapsed 2m 50s (remain 12m 9s) Loss: 0.1231(0.1009) LR: 0.00000349  
Epoch: [2][140/639] Elapsed 3m 18s (remain 11m 40s) Loss: 0.1079(0.1007) LR: 0.00000329  
Epoch: [2][160/639] Elapsed 3m 46s (remain 11m 11s) Loss: 0.0687(0.1016) LR: 0.00000309  
Epoch: [2][180/639] Elapsed 4m 13s (remain 10m 40s) Loss: 0.0824(0.1009) LR: 0.00000291  
Epoch: [2][200/639] Elapsed 4m 41s (remain 10m 12s) Loss: 0.0629(0.0996) LR: 0.00000273  
Epoch: [2][220/639]

Epoch 2 - avg_train_loss: 0.0939  avg_val_loss: 0.1060  time: 948s
INFO:__main__:Epoch 2 - avg_train_loss: 0.0939  avg_val_loss: 0.1060  time: 948s
Epoch 2 - Score: 0.4614  Scores: [0.4916481803556561, 0.45791177580881565, 0.4321726504608821, 0.4754099310574484, 0.46350073551204546, 0.4478174881181621]
INFO:__main__:Epoch 2 - Score: 0.4614  Scores: [0.4916481803556561, 0.45791177580881565, 0.4321726504608821, 0.4754099310574484, 0.46350073551204546, 0.4478174881181621]
Epoch 2 - Save Best Score: 0.4614 Model
INFO:__main__:Epoch 2 - Save Best Score: 0.4614 Model


...

Epoch 2 - avg_train_loss: 0.0927  avg_val_loss: 0.1013  time: 951s
INFO:__main__:Epoch 2 - avg_train_loss: 0.0927  avg_val_loss: 0.1013  time: 951s
Epoch 2 - Score: 0.4506  Scores: [0.48328408550984286, 0.44267724983812107, 0.40656913525201777, 0.450641636894226, 0.4692348531082845, 0.4509382648783194]
INFO:__main__:Epoch 2 - Score: 0.4506  Scores: [0.48328408550984286, 0.44267724983812107, 0.40656913525201777, 0.450641636894226, 0.4692348531082845, 0.4509382648783194]
Epoch 2 - Save Best Score: 0.4506 Model
INFO:__main__:Epoch 2 - Save Best Score: 0.4506 Model


Epoch: [3][0/642] Elapsed 0m 1s (remain 20m 28s) Loss: 0.1211(0.1211) LR: 0.00000083  
Epoch: [3][20/642] Elapsed 0m 30s (remain 14m 57s) Loss: 0.0899(0.0891) LR: 0.00000080  
Epoch: [3][40/642] Elapsed 0m 58s (remain 14m 20s) Loss: 0.0831(0.0917) LR: 0.00000078  
Epoch: [3][60/642] Elapsed 1m 26s (remain 13m 42s) Loss: 0.0956(0.0899) LR: 0.00000076  
Epoch: [3][80/642] Elapsed 1m 54s (remain 13m 11s) Loss: 0.0703(0.0902) LR: 0.00000075  
Epoch: [3][100/642] Elapsed 2m 22s (remain 12m 42s) Loss: 0.0640(0.0906) LR: 0.00000073  
Epoch: [3][120/642] Elapsed 2m 51s (remain 12m 17s) Loss: 0.1461(0.0905) LR: 0.00000072  
Epoch: [3][140/642] Elapsed 3m 18s (remain 11m 43s) Loss: 0.1181(0.0905) LR: 0.00000072  
Epoch: [3][160/642] Elapsed 3m 46s (remain 11m 15s) Loss: 0.1132(0.0910) LR: 0.00000071  
Epoch: [3][180/642] Elapsed 4m 14s (remain 10m 47s) Loss: 0.0466(0.0896) LR: 0.00000071  
Epoch: [3][200/642] Elapsed 4m 41s (remain 10m 17s) Loss: 0.0834(0.0882) LR: 0.00000070  
Epoch: [3][220/64

Epoch 3 - avg_train_loss: 0.0883  avg_val_loss: 0.1007  time: 949s
INFO:__main__:Epoch 3 - avg_train_loss: 0.0883  avg_val_loss: 0.1007  time: 949s
Epoch 3 - Score: 0.4491  Scores: [0.4805314933898941, 0.44016505072500084, 0.4052084486872293, 0.45127862146918724, 0.4685919589906107, 0.44858643204793397]
INFO:__main__:Epoch 3 - Score: 0.4491  Scores: [0.4805314933898941, 0.44016505072500084, 0.4052084486872293, 0.45127862146918724, 0.4685919589906107, 0.44858643204793397]
Epoch 3 - Save Best Score: 0.4491 Model
INFO:__main__:Epoch 3 - Save Best Score: 0.4491 Model


Epoch: [4][0/642] Elapsed 0m 1s (remain 19m 47s) Loss: 0.1130(0.1130) LR: 0.00000070  
Epoch: [4][20/642] Elapsed 0m 29s (remain 14m 34s) Loss: 0.0781(0.0934) LR: 0.00000070  
Epoch: [4][40/642] Elapsed 0m 56s (remain 13m 49s) Loss: 0.1001(0.0923) LR: 0.00000070  
Epoch: [4][60/642] Elapsed 1m 24s (remain 13m 23s) Loss: 0.0597(0.0920) LR: 0.00000070  
Epoch: [4][80/642] Elapsed 1m 52s (remain 12m 57s) Loss: 0.0912(0.0897) LR: 0.00000070  
Epoch: [4][100/642] Elapsed 2m 19s (remain 12m 28s) Loss: 0.0997(0.0910) LR: 0.00000070  
Epoch: [4][120/642] Elapsed 2m 46s (remain 11m 58s) Loss: 0.1153(0.0912) LR: 0.00000070  
Epoch: [4][140/642] Elapsed 3m 15s (remain 11m 33s) Loss: 0.0969(0.0903) LR: 0.00000070  
Epoch: [4][160/642] Elapsed 3m 42s (remain 11m 4s) Loss: 0.1000(0.0907) LR: 0.00000070  
Epoch: [4][180/642] Elapsed 4m 9s (remain 10m 36s) Loss: 0.0660(0.0893) LR: 0.00000070  
Epoch: [4][200/642] Elapsed 4m 38s (remain 10m 10s) Loss: 0.0800(0.0896) LR: 0.00000070  
Epoch: [4][220/642]

Epoch 4 - avg_train_loss: 0.0879  avg_val_loss: 0.1008  time: 952s
INFO:__main__:Epoch 4 - avg_train_loss: 0.0879  avg_val_loss: 0.1008  time: 952s
Epoch 4 - Score: 0.4494  Scores: [0.4813435995932851, 0.440687352736938, 0.4055690907576167, 0.4496107505924214, 0.46928728382978285, 0.45015120044668927]
INFO:__main__:Epoch 4 - Score: 0.4494  Scores: [0.4813435995932851, 0.440687352736938, 0.4055690907576167, 0.4496107505924214, 0.46928728382978285, 0.45015120044668927]
========== fold: 4 result ==========


In [ ]:
runtime.unassign()